In [1]:
import pandas as pd
import numpy as np

#Image show
import plotly.express as px
from skimage import io as Io
import plotly.graph_objects as go

#Spotify
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Color detection
from colorthief import ColorThief
import sys
from urllib.request import urlopen
import io

spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id="7d8caadb2b764965be0bb703bfd4e2c2",
                                                           client_secret="59db75fa692b4feb84e031c461fd1720"))

In [50]:
#Retrieving: Name, uri, Album image, and parameteres from 100 songs from 10 different genres
genres = ['anime','bossanova','chill','drum-and-bass','dubstep','jazz','post-dubstep','sad','soul','reggaeton']
recommendations = spotify.recommendations(seed_genres=['dubstep'], limit=100) # get 100 songs
img = Io.imread(recom['tracks'][0]['album']['images'][0]['url'])
parameters = spotify.audio_features(recom['tracks'][0]['uri'])

print(recom['tracks'][0]['name']) #Song Name
print(recom['tracks'][0]['uri']) #Song uri
print(img) #Image [array]
print(list(parameters[0].items())[:11]) #Parameters of the song

Like a Boss
spotify:track:5MhqNNZnkamSpD55bx7r6n
[[[10 10 10]
  [ 9  9  9]
  [ 8  8  8]
  ...
  [ 7  7  7]
  [ 7  7  7]
  [ 7  7  7]]

 [[ 9  9  9]
  [ 8  8  8]
  [ 7  7  7]
  ...
  [ 7  7  7]
  [ 7  7  7]
  [ 7  7  7]]

 [[ 7  7  7]
  [ 7  7  7]
  [ 6  6  6]
  ...
  [ 7  7  7]
  [ 7  7  7]
  [ 7  7  7]]

 ...

 [[ 3  3  3]
  [ 2  2  2]
  [ 1  1  1]
  ...
  [ 2  2  2]
  [ 2  2  2]
  [ 2  2  2]]

 [[ 2  2  2]
  [ 1  1  1]
  [ 1  1  1]
  ...
  [ 1  1  1]
  [ 1  1  1]
  [ 1  1  1]]

 [[ 3  3  3]
  [ 2  2  2]
  [ 1  1  1]
  ...
  [ 2  2  2]
  [ 2  2  2]
  [ 2  2  2]]]
[('danceability', 0.587), ('energy', 0.92), ('key', 11), ('loudness', -2.251), ('mode', 1), ('speechiness', 0.0736), ('acousticness', 0.00103), ('instrumentalness', 0.583), ('liveness', 0.102), ('valence', 0.635), ('tempo', 70.017)]


In [47]:
songs = pd.DataFrame(columns=['name','uri','image','dominant_color','color_palette','parameters']) #list for 100 songs
genres = ['anime','bossanova','chill','drum-and-bass','dubstep','jazz','post-dubstep','sad','soul','reggaeton']
recommendations = spotify.recommendations(seed_genres=['dubstep'], limit=100) # get 100 songs

for song in recommendations['tracks']:
    parameters = spotify.audio_features(song['uri'])[0] #get song parameters
    keys_to_remove = ['type','id','uri','track_href','analysis_url','time_signature']
    for key in keys_to_remove:
        parameters.pop(key)
    
    img_url = song['album']['images'][0]['url']
    img = Io.imread(img_url) #get song cover
    
    fd = urlopen(img_url)
    f = io.BytesIO(fd.read())
    color_thief = ColorThief(f)
    dominant_color = np.array(color_thief.get_color(quality=1)).reshape(1,1,3) #get dominant color
    palette = np.array(color_thief.get_palette(color_count=6)).reshape(2,3,3) # get color palette
    
    reg = {'name': song['name'], #create dict
            'uri': song['uri'],
            'image': img,
            'dominant_color': dominant_color,
            'color_palette': palette,
            'parameters': parameters}
    
    songs = songs.append(reg, ignore_index =True) #add song to list
    
dfparameters = pd.json_normalize(songs['parameters'])
songs.drop(['parameters'], axis =1, inplace=True)
songs = songs.join(dfparameters)
display(songs.head())

,name,uri,image,dominant_color,color_palette,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,Rubadub Shakedown,spotify:track:5BJoVbcufI9m0V8AzwBfok,"[[[155, 130, 136], [160, 133, 140], [181, 152,...","[[[230, 209, 184]]]","[[[36, 30, 20], [231, 212, 187], [129, 78, 65]...",0.871,0.397,11,-5.482,0,0.2120,0.019200,0.008750,0.0940,0.6410,88.377,218240
1,Epidemic - Original,spotify:track:5Gx3gMjsfZOEdVZwnrjwVJ,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...","[[[4, 38, 4]]]","[[[45, 194, 10], [4, 38, 4], [202, 238, 172]],...",0.734,0.954,8,-5.199,0,0.0807,0.332000,0.114000,0.2600,0.1870,111.998,277379
2,Delicious - Vocal Mix,spotify:track:2pl50hqbuARJ1FNPpAaVyG,"[[[0, 1, 22], [0, 1, 22], [0, 1, 22], [0, 1, 2...","[[[31, 30, 49]]]","[[[230, 207, 184], [24, 25, 46], [115, 78, 78]...",0.484,0.950,4,-4.005,0,0.0520,0.001720,0.000047,0.0756,0.3570,174.973,304869
3,Lights - Bassnectar Remix,spotify:track:3woWcxPt1HWi2KiTptZlDe,"[[[140, 131, 148], [124, 164, 156], [110, 136,...","[[[132, 93, 75]]]","[[[131, 113, 89], [37, 49, 48], [205, 217, 185...",0.538,0.813,1,-3.831,1,0.0301,0.000286,0.841000,0.0870,0.1740,140.007,303429
4,Put Ya Hands Up feat. Roly MC,spotify:track:2hJtyFIvRoEdQO0iothupV,"[[[168, 170, 169], [167, 169, 168], [164, 166,...","[[[23, 42, 42]]]","[[[206, 214, 214], [14, 35, 35], [70, 180, 178...",0.504,0.990,11,-1.584,0,0.0669,0.003770,0.887000,0.3330,0.0555,140.015,341143
